This notebook will guide to guide you through accquiring data with Cosi Measure without a GUI

# 1. Creating Pathfiles

Here we create a file with a list of points, named pathfile. You can visualize the pathfile with `postprocessing/visualizer/plot_3d_snake.py`. 

This example will map the $OSI^2One$ V1 Magnet with a ball-path. It will find the center of the magnet bore based on some reference points that you need to enter manually, see code block below.

In [ ]:
import numpy as np
import os
from devices.cosi import CosiMeasure
from devices.gaussmeter import gaussmeter
from utils.osi2_positioning import get_magnet_center_abs
from pathfiles.ball_path import ball_path
from pathfiles.extrempoints_path import create_extrempoints_pathfile
from utils.utils import get_formatted_datetime

descriptor = '_describe-your-measurement-here'

directory = "_my_data/" + get_formatted_datetime() + descriptor + '/'

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# do not change the following file names for the GUI shimming and fieldviewer.
filename_ball_path = directory + 'ballpath.txt'
filename_raw_dat = directory + 'raw_dat.csv'
filename_extrempoints_pathfile = directory + 'extrem_points.txt' 
filename_extrem_points_before = directory + 'extrempoints_raw_dat_before.csv' 
filename_extrem_points_after = directory + 'extrempoints_raw_dat_after.csv'
filename_centerpoint = directory + 'centerpoint.csv'

# measure this points in your setup before
pr = np.array([2.0, 417.0, 270.0])      # ref point right aka screw
pl = np.array([462.0, 415.0, 280.0])    # screw left
pt = np.array([235.0, 416.0, 505.0])    # screw top
center_point = get_magnet_center_abs(pr, pl, pt) # only tested for OSI2-One V1. other magnets might differ. 

np.savetxt(filename_centerpoint, center_point)

print(center_point)

# create path_files
radius = 85 # change to your needs
radius_npoints = 10 # change to your needs. 

t_measure = 0.6 # waiting time for filling the lakeshore filter window with new measurement values

ball = ball_path(filename_ball_path, center_point_input=center_point, radius_input=radius, radius_npoints_input=radius_npoints)
create_extrempoints_pathfile(center_point, radius, filename_extrempoints_pathfile)

# Init Cosi

In [ ]:
cosi = CosiMeasure()
is_colliding = cosi.check4bounds(filename_ball_path)
has_duplicates = cosi.check4duplicates(filename_ball_path)

# Homing Cosi

In [ ]:
print(cosi.home_y()) # moves y outside of the bore to y460

In [ ]:
print(cosi.home_x()) # x0
print(cosi.home_z()) # z0

# Move to Magnet Center

In [ ]:
print(cosi.move_to(y=cosi.max_y)) # safe in y axis
print(cosi.move_to(x=center_point[0], z=center_point[2])) # move to center in xz-plane
print(cosi.move_to(y=center_point[1]))

# Init Gaussmeter

Some settings of the gaussmeter are not yet implemented into this code. For example, one axis can be set to a higher resolution, it makes sense to set this into the direction of B0. This must be done by hand on the device for now.

In [ ]:
lakeshore = gaussmeter()
lakeshore.set_mode(mode='fast');
lakeshore.set_filter_on(True);
lakeshore.set_filter_window(1);
lakeshore.set_filter_points(5);
lakeshore.t_measure = t_measure

# Measure 

## 1. Extrem Points Before Mapping

In [ ]:
cosi.measure_along_path(filename_extrempoints_pathfile, filename_extrem_points_before, probe=lakeshore)

## 2. Mapping

In [ ]:
cosi.measure_along_path(filename_ball_path, filename_raw_dat, probe=lakeshore)

## 3. Extrem Points after Mapping

In [ ]:

cosi.measure_along_path(filename_extrempoints_pathfile, filename_extrem_points_after, probe=lakeshore)